> ### Import Library

In [ ]:
pip install pandas-gbq google-cloud-bigquery

In [ ]:
pip install google-cloud-bigquery==3.22.0 google-cloud-core==2.3.3


In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
from google.cloud import bigquery
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

PROJECT = "stellar-orb-451904-d9"
DATASET = "dashboard"
TABLE = "epus_pelayanan_assessment"
LOCATION = "asia-southeast2"
START_DATE = "2025-12-08"
END_DATE = "2025-12-14"

PKM_TARGETS = [     "Puskesmas Bobotsari",     "Puskesmas Serayu Larangan",     "Puskesmas Karang Tengah",     "Puskesmas Kaligondang",     "Puskesmas Bojong",     "Puskesmas Karangjambu",     "Puskesmas Bojongsari",     "Puskesmas Karanganyar",     "Puskesmas Purbalingga",     "Puskesmas Padamara",     "Puskesmas Rembang",     "Puskesmas Pengadegan"  ]

client = bigquery.Client(project=PROJECT, location=LOCATION)

sql = f"""
SELECT *
FROM `{PROJECT}.{DATASET}.{TABLE}`
WHERE DATE(tanggal_catat) BETWEEN '{START_DATE}' AND '{END_DATE}'
AND puskesmas_names IN UNNEST(@pkm_targets)
"""

job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ArrayQueryParameter("pkm_targets", "STRING", PKM_TARGETS)
    ]
)

data = client.query(sql, job_config=job_config).to_dataframe()
print(f"✅ Data loaded: {len(data):,} rows for {len(PKM_TARGETS)} puskesmas ({START_DATE} to {END_DATE})")
print("Sample columns:", data.columns.tolist()[:10])
print("Sample petugas:", data['perawat_bidan_nutrisionist_sanitarian'].dropna().unique()[:5])


✅ Data loaded: 7,813 rows for 12 puskesmas (2025-12-08 to 2025-12-14)
Sample columns: ['tgl_lahir_valid', 'tanggal_catat', 'file_name', 'tanggal_periksa', 'tgllahir', 'nomor_telp', 'puskesmas_names', 'id', 'jkel', 'height_cm']
Sample petugas: ['AGUS BUDIANTO' 'ANJARSARI' 'ARI ABRIYANTI' 'ARIANI NURDIATI'
 'ARIF BUDIARSO']


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7813 entries, 0 to 7812
Data columns (total 90 columns):
 #   Column                                     Non-Null Count  Dtype         
---  ------                                     --------------  -----         
 0   tgl_lahir_valid                            7773 non-null   boolean       
 1   tanggal_catat                              7813 non-null   datetime64[us]
 2   file_name                                  7813 non-null   object        
 3   tanggal_periksa                            7717 non-null   datetime64[us]
 4   tgllahir                                   7773 non-null   dbdate        
 5   nomor_telp                                 7376 non-null   object        
 6   puskesmas_names                            7813 non-null   object        
 7   id                                         7515 non-null   object        
 8   jkel                                       7813 non-null   object        
 9   height_cm          

> #### Missing Values

In [ ]:
import pandas as pd
import numpy as np

def make_missing_summary(data, value_col="sys", hcw_col="perawat_bidan_nutrisionist_sanitarian"):
    """
    Versi Python dari summary_table (R).
    Menghitung jumlah & persentase nilai yang ada & hilang per petugas.
    Mengembalikan hanya kolom utama:
    hcw, <var>_present, <var>_present_pct, <var>_na, <var>_na_pct
    """
    df = data.copy()

    # Normalisasi nilai petugas
    df[hcw_col] = df[hcw_col].replace(['', ' ', 'None', 'none', 'NaN', 'nan'], np.nan)

    # Hitung present & missing
    df["present"] = np.where(df[value_col].notna(), 1, 0)
    df["missing"] = np.where(df[value_col].isna(), 1, 0)

    # Rekap per petugas
    summary = (
        df.groupby(hcw_col, dropna=False)
        .agg(present=("present", "sum"), missing=("missing", "sum"))
        .reset_index()
    )

    # Hitung total & persentase
    summary["total"] = summary["present"] + summary["missing"]
    summary["present_pct"] = round(100 * summary["present"] / summary["total"], 1)
    summary["missing_pct"] = round(100 * summary["missing"] / summary["total"], 1)

    # Urutkan dari missing terbanyak
    summary = summary.sort_values("missing", ascending=False).reset_index(drop=True)

    # Rename kolom sesuai konvensi
    summary = summary.rename(columns={
        hcw_col: "hcw",
        "present": f"{value_col}_present",
        "missing": f"{value_col}_na",
        "present_pct": f"{value_col}_present_pct",
        "missing_pct": f"{value_col}_na_pct"
    })

    # Hanya tampilkan kolom utama
    summary = summary[["hcw",
                       f"{value_col}_present",
                       f"{value_col}_present_pct",
                       f"{value_col}_na",
                       f"{value_col}_na_pct"]]

    return summary


In [ ]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import shutil
import numpy as np

# ===============================================================
# CONFIGURATION
# ===============================================================
COL_PKM = "puskesmas_names"
COL_PETUGAS = "perawat_bidan_nutrisionist_sanitarian"

VARS = ["sys", "dia", "heart_rate", "weight_kg", "height_cm", "resp_rate_per_minute","body_temp_c"]

PUSKESMAS_LIST = [     "Puskesmas Bobotsari",     "Puskesmas Serayu Larangan",     "Puskesmas Karang Tengah",     "Puskesmas Kaligondang",     "Puskesmas Bojong",     "Puskesmas Karangjambu",     "Puskesmas Bojongsari",     "Puskesmas Karanganyar",     "Puskesmas Purbalingga",     "Puskesmas Padamara",     "Puskesmas Rembang",     "Puskesmas Pengadegan"  ]

OUTPUT_DIR = "Visualization of missing values"
os.makedirs(OUTPUT_DIR, exist_ok=True)


# ===============================================================
# HELPER FUNCTION (from you)
# ===============================================================
def make_missing_summary(data, value_col="sys", hcw_col="perawat_bidan_nutrisionist_sanitarian"):
    """
    Versi Python dari summary_table (R).
    Menghitung jumlah & persentase nilai yang ada & hilang per petugas.
    Mengembalikan hanya kolom utama:
    hcw, <var>_present, <var>_present_pct, <var>_na, <var>_na_pct
    """
    df = data.copy()
    df[hcw_col] = df[hcw_col].replace(['', ' ', 'None', 'none', 'NaN', 'nan'], np.nan)

    # Hitung present & missing
    df["present"] = np.where(df[value_col].notna(), 1, 0)
    df["missing"] = np.where(df[value_col].isna(), 1, 0)

    # Rekap per petugas
    summary = (
        df.groupby(hcw_col, dropna=False)
        .agg(present=("present", "sum"), missing=("missing", "sum"))
        .reset_index()
    )

    # Hitung total & persentase
    summary["total"] = summary["present"] + summary["missing"]
    summary["present_pct"] = round(100 * summary["present"] / summary["total"], 1)
    summary["missing_pct"] = round(100 * summary["missing"] / summary["total"], 1)

    # Urutkan dari missing terbanyak
    summary = summary.sort_values("missing", ascending=False).reset_index(drop=True)

    # Rename kolom sesuai konvensi
    summary = summary.rename(columns={
        hcw_col: "hcw",
        "present": f"{value_col}_present",
        "missing": f"{value_col}_na",
        "present_pct": f"{value_col}_present_pct",
        "missing_pct": f"{value_col}_na_pct"
    })

    # Hanya tampilkan kolom utama
    summary = summary[[
        "hcw",
        f"{value_col}_present",
        f"{value_col}_present_pct",
        f"{value_col}_na",
        f"{value_col}_na_pct"
    ]]
    return summary


# ===============================================================
# PLOT FUNCTION (compatible with make_missing_summary)
# ===============================================================
def plot_missing_summary(data, pkm_name, value_col, hcw_col):
    """Generate and save missing vs non-missing visualization for one Puskesmas and one variable."""
    if pkm_name == "All puskesmas":
        df = data.copy()
    else:
        df = data[data[COL_PKM] == pkm_name].copy()

    if df.empty:
        print(f"[SKIP] No data available for {pkm_name} – {value_col}")
        return

    # 🔹 Gunakan fungsi buatanmu
    summary = make_missing_summary(df, value_col, hcw_col)
    if summary.empty:
        print(f"[SKIP] {pkm_name} – {value_col}: summary is empty")
        return

    # Tambahkan total untuk hitung proporsi
    summary[f"{value_col}_total"] = summary[f"{value_col}_present"] + summary[f"{value_col}_na"]
    summary["hcw_short"] = summary["hcw"].astype(str).apply(lambda x: x[:12])

    # Long format untuk seaborn
    summary_long = summary.melt(
        id_vars=["hcw", "hcw_short", f"{value_col}_total"],
        value_vars=[f"{value_col}_present", f"{value_col}_na"],
        var_name="status",
        value_name="count"
    )

    summary_long["status"] = summary_long["status"].replace({
        f"{value_col}_present": "Non-Missing Values",
        f"{value_col}_na": "Missing Values"
    })

    # ===============================================================
    # PLOT
    # ===============================================================
    plt.figure(figsize=(20, 45))
    ax = sns.barplot(
        data=summary_long,
        y="hcw_short",
        x="count",
        hue="status",
        palette=["#4CAF50", "#F44336"],
        orient="h"
    )

    plt.title(
        f"Distribution of {value_col.replace('_', ' ').title()} Data per Health Worker – {pkm_name}",
        fontsize=20, pad=20
    )
    plt.ylabel("Health Worker", fontsize=14)
    plt.xlabel("Number of Observations", fontsize=14)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)

    plt.legend(
        title="Data Status",
        title_fontsize=18,
        fontsize=16,
        bbox_to_anchor=(1.05, 1),
        loc="upper left"
    )

    # ===============================================================
    # LABEL (jumlah + persentase)
    # ===============================================================
    for container in ax.containers:
        for bar in container:
            width = bar.get_width()
            y_center = bar.get_y() + bar.get_height() / 2

            # Ambil label pekerja berdasarkan urutan
            idx = int(round(y_center))
            if idx < len(ax.get_yticklabels()):
                hcw_label = ax.get_yticklabels()[idx].get_text()
            else:
                hcw_label = None

            # Cari total untuk petugas tsb
            total = summary_long.loc[summary_long["hcw_short"] == hcw_label, f"{value_col}_total"]
            total_val = total.iloc[0] if not total.empty else 0
            pct = 100 * width / total_val if total_val > 0 else 0

            # Tulis label di kanan bar
            ax.text(width + 2, y_center, f"{int(width)} ({pct:.1f}%)",
                    ha="left", va="center", fontsize=9)

    plt.tight_layout(rect=[0, 0, 0.85, 1])

    # ===============================================================
    # SAVE PDF
    # ===============================================================
    save_dir = os.path.join(OUTPUT_DIR, pkm_name)
    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, f"{value_col}.pdf")

    plt.savefig(save_path, format="pdf", bbox_inches="tight")
    plt.close()
    print(f"[OK] {pkm_name} – {value_col} saved to {save_path}")


# ===============================================================
# MAIN PIPELINE
# ===============================================================
for pkm in PUSKESMAS_LIST + ["All puskesmas"]:
    for var in VARS:
        plot_missing_summary(data, pkm_name=pkm, value_col=var, hcw_col=COL_PETUGAS)

print("\n✅ Semua visualisasi missing values selesai dibuat!")


# ===============================================================
# ZIP OUTPUT
# ===============================================================
zip_path = f"{OUTPUT_DIR}.zip"
shutil.make_archive(OUTPUT_DIR, 'zip', OUTPUT_DIR)
print(f"\n📦 Semua hasil telah dikompres ke: {zip_path}")
print("Siap untuk diunduh!")


> ### Heaping, Heatmap

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# ===============================================================
# 🔹 Configuration
# ===============================================================
COL_PKM = "puskesmas_names"
COL_PETUGAS = "perawat_bidan_nutrisionist_sanitarian"

VARS = ["sys", "dia", "heart_rate", "weight_kg", "height_cm", "resp_rate_per_minute","body_temp_c"]

PUSKESMAS_LIST = [     "Puskesmas Bobotsari",     "Puskesmas Serayu Larangan",     "Puskesmas Karang Tengah",     "Puskesmas Kaligondang",     "Puskesmas Bojong",     "Puskesmas Karangjambu",     "Puskesmas Bojongsari",     "Puskesmas Karanganyar",     "Puskesmas Purbalingga",     "Puskesmas Padamara",     "Puskesmas Rembang",     "Puskesmas Pengadegan"  ]

OUTPUT_DIR = "Visualization of last-digit distribution"
os.makedirs(OUTPUT_DIR, exist_ok=True)


# ===============================================================
# 🔹 Function: Compute last-digit distribution per health worker
# ===============================================================
def make_lastdigit_summary(data, value_col="height_cm", hcw_col=COL_PETUGAS):
    """
    Compute the last-digit (0–9) distribution of numeric values per health worker.
    If decimal values exist (e.g., 65.6), the last digit of the rounded integer is used.
    """
    df = data.copy()
    df[hcw_col] = df[hcw_col].replace(['', ' ', 'None', 'none', 'NaN', 'nan'], np.nan)

    # Ensure numeric
    df[value_col] = pd.to_numeric(df[value_col], errors="coerce")

    # Extract last digit (0–9)
    def get_last_digit(x):
        if pd.isna(x):
            return np.nan
        return int(str(int(round(x)))[-1])

    df["last_digit"] = df[value_col].apply(get_last_digit)
    df = df.dropna(subset=["last_digit", hcw_col])

    # Count per worker × digit
    summary = (
        df.groupby([hcw_col, "last_digit"])
        .size()
        .reset_index(name="count")
    )

    # Compute total per worker
    total = summary.groupby(hcw_col)["count"].sum().reset_index(name="total")
    summary = summary.merge(total, on=hcw_col, how="left")

    # Compute percentage
    summary["pct"] = round(100 * summary["count"] / summary["total"], 1)

    # Clean names
    summary = summary.rename(columns={hcw_col: "hcw"})
    summary["hcw"] = summary["hcw"].fillna("NaN")
    summary["hcw_short"] = summary["hcw"].apply(lambda x: x[:10] if isinstance(x, str) else "NaN")

    return summary


# ===============================================================
# 🔹 Function: Plot and Save Heatmap
# ===============================================================
def plot_lastdigit_heatmap(data, pkm_name, value_col, hcw_col):
    """Create and save last-digit distribution heatmap for a given Puskesmas and variable."""
    if pkm_name == "All puskesmas":
        data_filtered = data.copy()
    else:
        data_filtered = data.query(f'{COL_PKM} == "{pkm_name}"').copy()

    if data_filtered.empty:
        print(f"[SKIP] No data available for {pkm_name} – {value_col}")
        return

    summary = make_lastdigit_summary(data_filtered, value_col=value_col, hcw_col=hcw_col)
    if summary.empty:
        print(f"[SKIP] No valid data for {pkm_name} – {value_col}")
        return

    # Pivot: workers on Y, digits on X
    pivot = summary.pivot(index="hcw_short", columns="last_digit", values="pct").fillna(0)

    # ===============================================================
    # 🔹 Plot heatmap
    # ===============================================================
    plt.figure(figsize=(25, 45))
    sns.heatmap(pivot, annot=True, fmt=".1f", cmap="flare", cbar_kws={'label': 'Proportion (%)'})

    plt.title(f"Distribution of Last Digits (0–9) for {value_col.replace('_', ' ').title()} – {pkm_name}",
              fontsize=20, pad=20)
    plt.xlabel("Last Digit (0–9)", fontsize=14)
    plt.ylabel("Health Worker (first 10 characters)", fontsize=14)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12, rotation=0)
    plt.tight_layout()

    # ===============================================================
    # 🔹 Save as PDF
    # ===============================================================
    save_dir = os.path.join(OUTPUT_DIR, pkm_name)
    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, f"{value_col}.pdf")
    plt.savefig(save_path, format="pdf", bbox_inches="tight")
    plt.close()

    print(f"[OK] {pkm_name} – {value_col} saved to {save_path}")


# ===============================================================
# 🔹 Main Loop: Each Puskesmas × Each Variable
# ===============================================================
for pkm in PUSKESMAS_LIST + ["All puskesmas"]:
    for var in VARS:
        plot_lastdigit_heatmap(data, pkm_name=pkm, value_col=var, hcw_col=COL_PETUGAS)


> ### Heaping, Histogram Last Digit!

In [ ]:
data.to_excel("text.xlsx")

In [ ]:
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shutil
from matplotlib.backends.backend_pdf import PdfPages

# ===============================================================
# CONFIGURATION
# ===============================================================
COL_PKM = "puskesmas_names"
COL_PETUGAS = "perawat_bidan_nutrisionist_sanitarian"

VARS = [
    "sys", "dia", "heart_rate", "weight_kg",
    "height_cm", "resp_rate_per_minute", "body_temp_c"
]

PUSKESMAS_LIST = [
    "Puskesmas Bobotsari",
    "Puskesmas Serayu Larangan",
    "Puskesmas Karang Tengah",
    "Puskesmas Kaligondang",
    "Puskesmas Bojong",
    "Puskesmas Karangjambu",
    "Puskesmas Bojongsari",
    "Puskesmas Karanganyar",
    "Puskesmas Purbalingga",
    "Puskesmas Padamara",
    "Puskesmas Rembang",
    "Puskesmas Pengadegan"
]
OUTPUT_DIR = "Visualization of last-digit histogram"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ===============================================================
# HELPERS
# ===============================================================
def normalize_worker(s: pd.Series) -> pd.Series:
    def clean_name(x):
        if pd.isna(x) or str(x).strip() == "":
            return "TIDAK ADA PETUGAS"
        x = str(x).upper().strip()
        x = re.sub(r"[\.,()]", "", x)
        x = re.sub(r"\s+", " ", x)
        return x
    return s.apply(clean_name)

def short_label(name: str) -> str:
    parts = str(name).title().split()
    return f"{parts[0]} {parts[1][:3]}" if len(parts) >= 2 else parts[0]

def make_lastdigit_distribution(df, value_col):
    df = df.copy()
    df[value_col] = pd.to_numeric(df[value_col], errors="coerce")
    df = df.dropna(subset=[value_col])
    if df.empty:
        return pd.DataFrame(columns=["last_digit", "count", "pct"]), 0

    if value_col == "body_temp_c":
        df["last_digit"] = ((df[value_col] * 10).astype(int) % 10)
    else:
        df["last_digit"] = df[value_col].round().astype(int) % 10

    summary = df["last_digit"].value_counts().sort_index().reset_index()
    summary.columns = ["last_digit", "count"]
    total = summary["count"].sum()
    summary["pct"] = 100 * summary["count"] / total
    return summary, total

# ===============================================================
# BASE HISTOGRAM
# ===============================================================
def plot_digit_distribution(summary, total, title, save_path):
    summary = summary.sort_values("last_digit").reset_index(drop=True)
    plt.figure(figsize=(12, 6))
    bars = plt.bar(summary["last_digit"], summary["pct"])
    plt.title(title, fontsize=16)
    plt.xlabel("Last Digit (0–9)")
    plt.ylabel("Percentage (%)")
    plt.ylim(0, 100)
    plt.xticks(range(10))

    for bar, row in zip(bars, summary.itertuples(index=False)):
        plt.text(
            bar.get_x() + bar.get_width() / 2,
            bar.get_height() + 1,
            f"{int(row.count)} / {int(total)} = {row.pct:.1f}%",
            ha="center", va="bottom", fontsize=9, rotation=90
        )

    plt.tight_layout()
    plt.savefig(save_path, format="pdf", bbox_inches="tight")
    plt.close()

# ===============================================================
# WORKER HEAPING (SINGLE PAGE)
# ===============================================================
def plot_worker_heaping(df_src, value_col, pkm_name, pdf, mode="percentage"):
    df = df_src[df_src[COL_PKM] == pkm_name].copy()
    df[value_col] = pd.to_numeric(df[value_col], errors="coerce")
    df = df.dropna(subset=[value_col])
    if df.empty:
        return

    df["full_name"] = normalize_worker(df[COL_PETUGAS])
    df["short_name"] = df["full_name"].apply(short_label)

    if value_col == "body_temp_c":
        df["last_digit"] = ((df[value_col] * 10).astype(int) % 10)
    else:
        df["last_digit"] = df[value_col].round().astype(int) % 10

    summary_pkm, _ = make_lastdigit_distribution(df, value_col)
    heaping_digits = summary_pkm.loc[summary_pkm["pct"] > 15, "last_digit"].tolist()
    if not heaping_digits:
        return

    totals = df.groupby("full_name")[value_col].size().astype(int)

    counts = (
        df[df["last_digit"].isin(heaping_digits)]
        .groupby(["full_name", "last_digit"])[value_col]
        .size()
        .reset_index(name="count")
    )

    counts["total_all"] = counts["full_name"].map(totals)
    counts["pct"] = 100 * counts["count"] / counts["total_all"]

    name_map = df.drop_duplicates("full_name").set_index("full_name")["short_name"].to_dict()

    worker_order = totals.sort_values(ascending=False).index.tolist()
    digits = sorted(heaping_digits)
    workers = [w for w in worker_order if w in counts["full_name"].unique()]

    n_workers = len(workers)
    n_digits = len(digits)
    bar_width = 0.8 / n_digits
    x_base = np.arange(n_workers)
    fig_h = max(6, min(12, 4 + 0.25 * n_workers))
    fig, ax = plt.subplots(figsize=(max(8, n_workers), fig_h))

    for j, d in enumerate(digits):
        x = x_base + (j - (n_digits - 1) / 2) * bar_width
        vals = []
        for w in workers:
            row = counts[(counts["full_name"] == w) & (counts["last_digit"] == d)]
            if row.empty:
                vals.append(0)
            else:
                vals.append(
                    row["pct"].values[0] if mode == "percentage"
                    else row["count"].values[0]
                )
        ax.bar(x, vals, width=bar_width, label=str(d))

    ax.set_title(
        f"Heaping digits {digits} for {value_col.replace('_',' ').title()} "
        f"— {pkm_name} {'by Percentage' if mode=='percentage' else 'by Total'}"
    )
    ax.set_ylabel(
        "Percentage within Health Worker (%)" if mode == "percentage" else "Total Count"
    )
    if mode == "percentage":
        ax.set_ylim(0, 100)

    ax.set_xticks(x_base)
    ax.set_xticklabels([name_map[w] for w in workers], rotation=45, ha="right")
    ax.legend(title="Digit")
    fig.tight_layout()
    pdf.savefig(fig)
    plt.close(fig)

# ===============================================================
# MAIN PIPELINE
# ===============================================================
for var in VARS:
    print(f"\n===== Processing variable: {var} =====")

    summary_all, total_all = make_lastdigit_distribution(data, var)
    if not summary_all.empty:
        save_dir_all = os.path.join(OUTPUT_DIR, "All puskesmas")
        os.makedirs(save_dir_all, exist_ok=True)
        plot_digit_distribution(
            summary_all,
            total_all,
            f"Overall Distribution of Last Digits (0–9) for {var.replace('_',' ').title()} – All Puskesmas",
            os.path.join(save_dir_all, f"{var}_overall.pdf")
        )

    for pkm in PUSKESMAS_LIST:
        df_pkm = data[data[COL_PKM] == pkm]
        if df_pkm.empty:
            continue

        summary_pkm, _ = make_lastdigit_distribution(df_pkm, var)
        if summary_pkm.empty:
            continue

        save_dir_pkm = os.path.join(OUTPUT_DIR, pkm)
        os.makedirs(save_dir_pkm, exist_ok=True)

        plot_digit_distribution(
            summary_pkm,
            summary_pkm["count"].sum(),
            f"Distribution of Last Digits (0–9) for {var.replace('_',' ').title()} – {pkm}",
            os.path.join(save_dir_pkm, f"{var}_overall.pdf")
        )

        if (summary_pkm["pct"] > 15).any():
            out_pdf = os.path.join(save_dir_pkm, f"{var}_heaping.pdf")
            with PdfPages(out_pdf) as pdf:
                plot_worker_heaping(data, var, pkm, pdf, "percentage")
                plot_worker_heaping(data, var, pkm, pdf, "total")

print("\n✅ Semua visualisasi selesai dibuat!")

# ===============================================================
# ZIP OUTPUT
# ===============================================================
shutil.make_archive(OUTPUT_DIR, "zip", OUTPUT_DIR)
print(f"📦 Output dizip ke {OUTPUT_DIR}.zip")


>### Heaping, Histogram for each Puskesmas

In [ ]:
# ====================================
# Import & Config
# ====================================
from google.cloud import bigquery
import pandas as pd, numpy as np, matplotlib.pyplot as plt, matplotlib.ticker as mticker
from matplotlib.backends.backend_pdf import PdfPages
import os, zipfile

PROJECT, DATASET, TABLE = "stellar-orb-451904-d9", "dashboard", "epus_pelayanan_assessment"
START_DATE, END_DATE = "2025-12-08", "2025-12-14"
OUTPUT_DIR, ZIP_PATH, X_TICK_STEP = "visualization_week_2", "Visualization_week_2.zip", 1

PKM_TARGETS = [
    "Puskesmas Bobotsari",
    "Puskesmas Serayu Larangan",
    "Puskesmas Karang Tengah",
    "Puskesmas Kaligondang",
    "Puskesmas Bojong",
    "Puskesmas Karangjambu",
    "Puskesmas Bojongsari",
    "Puskesmas Karanganyar",
    "Puskesmas Purbalingga",
    "Puskesmas Padamara",
    "Puskesmas Rembang",
    "Puskesmas Pengadegan"
]

COL_DATE="tanggal_catat"; COL_PKM="puskesmas_names"
COL_HEIGHT="height_cm"; COL_WEIGHT="weight_kg"; COL_ABD="abd_circum_cm"
COL_SYS="sys"; COL_DIA="dia"; COL_RR="resp_rate_per_minute"; COL_HR="heart_rate"; COL_TEMP="body_temp_c"

VAR_UNITS = {COL_HEIGHT:"cm", COL_WEIGHT:"kg", COL_ABD:"cm", COL_SYS:"mmHg", COL_DIA:"mmHg",
             COL_RR:"/min", COL_HR:"bpm", COL_TEMP:"°C"}

plt.rcParams.update({"font.size":13,"axes.titlesize":15,"axes.labelsize":14,"xtick.labelsize":11,"ytick.labelsize":12,"legend.fontsize":12,"figure.titlesize":18})

# ====================================
# Load BigQuery
# ====================================
client = bigquery.Client(project=PROJECT)
query_str = f"""
SELECT * FROM `{PROJECT}.{DATASET}.{TABLE}`
WHERE DATE(tanggal_catat) BETWEEN '{START_DATE}' AND '{END_DATE}'
AND puskesmas_names IN UNNEST(@pkm_targets)
"""
job_config = bigquery.QueryJobConfig(query_parameters=[bigquery.ArrayQueryParameter("pkm_targets","STRING",PKM_TARGETS)])
print(f"🚀 Querying data {START_DATE} → {END_DATE} ...")
data = client.query(query_str, job_config=job_config).result().to_dataframe()
print(f"✅ {len(data):,} rows")

# Clean
data[COL_DATE] = pd.to_datetime(data[COL_DATE], errors="coerce")
for c in [COL_HEIGHT,COL_WEIGHT,COL_ABD,COL_SYS,COL_DIA,COL_RR,COL_HR,COL_TEMP]:
    if c in data.columns: data[c] = pd.to_numeric(data[c], errors="coerce")
df = data[data[COL_PKM].isin(PKM_TARGETS)].copy()

# ====================================
# Helpers
# ====================================
def _make_bins(s: pd.Series, bin_size: float, var: str):
    s = s.dropna()
    if s.empty: return np.arange(0,1,bin_size)
    if var == COL_TEMP:
        vmin, vmax = max(33, np.floor(s.min()*10)/10), min(43, np.ceil(s.max()*10)/10)
        if vmin==vmax: vmax += 0.1
        return np.arange(vmin, vmax+0.1, 0.1)
    vmin, vmax = np.floor(s.min()/bin_size)*bin_size, np.ceil(s.max()/bin_size)*bin_size
    if vmin==vmax: vmax += bin_size
    return np.arange(vmin, vmax+bin_size, bin_size)

def _fmt(v, var, is_int): return (f"{v:.0f}" if is_int else f"{v:.1f}") + VAR_UNITS.get(var,"")

def _bins_over_threshold(counts, edges, total, thr_pct, is_int=False):
    out=[]
    for i,c in enumerate(counts):
        pct = c/total*100 if total else 0
        if pct>thr_pct:
            val = int(edges[i]) if is_int else round((edges[i]+edges[i+1])/2,1)
            out.append((val,c,pct))
    return sorted(out,key=lambda x:x[1],reverse=True)

def _outlier_bins(series, lower, upper, bin_size, var, is_int=False, max_rows=60):
    s=series.dropna(); outs=s[(s<lower)|(s>upper)]
    if outs.empty: return []
    bins=_make_bins(outs,bin_size,var); counts,edges=np.histogram(outs,bins=bins)
    total=len(series); out=[]
    for i,c in enumerate(counts):
        if c==0: continue
        pct=c/total*100; val=int(edges[i]) if is_int else round((edges[i]+edges[i+1])/2,1)
        out.append((val,c,pct))
    return sorted(out,key=lambda x:x[1],reverse=True)[:max_rows]

def _panel_text(ax_text, var, is_int, top_bins, out_bins):
    """Tampilkan teks 'More than 15%' dan 'Outliers' dengan format rapi dan 2 kolom jika panjang."""
    ax_text.set_axis_off()
    y, dy = 0.98, 0.035

    if top_bins:
        ax_text.text(0.00, y, "More than 15%:", fontsize=11, fontweight="bold", va="top", ha="left")
        y -= 0.055
        for i, (v, c, p) in enumerate(top_bins, 1):
            ax_text.text(
                0.00, y,
                f"{i}. {_fmt(v, var, is_int)}, Count={c}, Percentage={p:.1f}%",
                fontsize=10, va="top", ha="left"
            )
            y -= dy
        y -= 0.02

    if out_bins:
        ax_text.text(0.00, y, "Outliers:", fontsize=11, fontweight="bold", va="top", ha="left")
        y -= 0.055

        # bagi 2 kolom kalau lebih dari 30
        left, right = out_bins[:30], out_bins[30:]
        y_left = y
        for i, (v, c, p) in enumerate(left, 1):
            ax_text.text(
                0.00, y_left,
                f"{i}. {_fmt(v, var, is_int)}, Count={c}, Percentage={p:.1f}%",
                fontsize=10, va="top", ha="left"
            )
            y_left -= dy

        if right:
            y_right = y
            for i, (v, c, p) in enumerate(right, 31):
                ax_text.text(
                    0.55, y_right,
                    f"{i}. {_fmt(v, var, is_int)}, Count={c}, Percentage={p:.1f}%",
                    fontsize=10, va="top", ha="left"
                )
                y_right -= dy


# ====================================
# Plot (histogram kiri + anotasi kanan)
# ====================================
def plot_weekly(df_in, pkm, var, start, end, clamp=None, IQR=1.5):
    d = df_in[df_in[COL_PKM]==pkm].dropna(subset=[var])
    if d.empty: return None

    # batas valid & bin
    if var==COL_RR: lower,upper,bs,is_int=10,30,1.0,True
    elif var==COL_TEMP: lower,upper,bs,is_int=33,43,0.1,False
    else:
        q1,q3=d[var].quantile([0.25,0.75]); iqr=q3-q1
        lower,upper=q1-1.5*iqr, q3+1.5*iqr; bs,is_int=1.0,True

    valid=d[(d[var]>=lower)&(d[var]<=upper)]
    if valid.empty: return None

    bins=_make_bins(valid[var],bs,var); counts,edges=np.histogram(valid[var],bins=bins)
    total=len(valid); pcts=counts/total*100
    cols=["#FF4C4C" if p>15 else "#4472C4" for p in pcts]

    fig,(ax,txt)=plt.subplots(1,2,figsize=(40,10),gridspec_kw={"width_ratios":[4.0,1.8]})
    centers=(edges[:-1]+edges[1:])/2 if not is_int else edges[:-1]
    ax.bar(centers,pcts,width=bs,align="center" if not is_int else "edge",edgecolor="black",color=cols)
    ax.set_title(f"{var} — {pkm}\nPeriode: {start} -> {end} (N={total})")
    ax.set_xlabel(f"{var} (bin size={bs})"); ax.set_ylabel("Percentage (%)")
    ax.grid(axis="y",linestyle=":",linewidth=0.8); ax.set_ylim(0,100)
    if clamp: ax.set_xlim(clamp[0]-bs/2, clamp[1]+bs/2)
    ax.xaxis.set_major_locator(mticker.MultipleLocator(X_TICK_STEP if is_int else 0.1))
    ax.xaxis.set_major_formatter(mticker.FormatStrFormatter('%.0f' if is_int else '%.1f'))
    plt.setp(ax.get_xticklabels(), rotation=90, ha="center", fontsize=9)

    top=_bins_over_threshold(counts,edges,total,15,is_int)
    outs=_outlier_bins(d[var],lower,upper,bs,var,is_int,60)
    _panel_text(txt,var,is_int,top,outs)

    fig.tight_layout()
    return fig

# ====================================
# Generate PDFs per puskesmas + ZIP all
# ====================================
if __name__ == "__main__":
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    vars_to_plot=[COL_HEIGHT,COL_WEIGHT,COL_ABD,COL_SYS,COL_DIA,COL_RR,COL_HR,COL_TEMP]
    generated=[]
    for pkm in PKM_TARGETS:
        pdf_path=os.path.join(OUTPUT_DIR,f"{pkm}.pdf".replace("/","_"))
        print(f"\n📄 Membuat PDF untuk {pkm} ...")
        dfp=df[df[COL_PKM]==pkm]
        if dfp.empty: print("  (kosong)"); continue

        # clamp_x per variabel (pakai data valid minggu ini)
        ranges={}
        for v in vars_to_plot:
            s=dfp[v].dropna()
            if s.empty: continue
            if v==COL_RR: s=s[(s>=10)&(s<=30)]
            elif v==COL_TEMP: s=s[(s>=33)&(s<=43)]
            else:
                q1,q3=s.quantile([0.25,0.75]); iqr=q3-q1
                s=s[(s>=q1-1.5*iqr)&(s<=q3+1.5*iqr)]
            if s.empty: continue
            vmin,vmax=np.floor(s.min()),np.ceil(s.max())
            if v==COL_TEMP:
                vmin=max(33, np.floor(s.min()*10)/10); vmin=float(vmin)
                vmax=min(43, np.ceil(s.max()*10)/10);  vmax=float(vmax)
            ranges[v]=(vmin,vmax)

        with PdfPages(pdf_path) as pdf:
            for v in vars_to_plot:
                if v not in ranges: continue
                fig=plot_weekly(df, pkm, v, START_DATE, END_DATE, clamp=ranges[v])
                if fig: pdf.savefig(fig); plt.close(fig)
        print(f"✅ Disimpan: {pdf_path}"); generated.append(pdf_path)

    # Zip semua PDF
    if generated:
        print(f"\n📦 Mengarsipkan {len(generated)} file ke {ZIP_PATH} ...")
        with zipfile.ZipFile(ZIP_PATH, "w", zipfile.ZIP_DEFLATED) as zf:
            for f in generated: zf.write(f, arcname=os.path.basename(f))
        print(f"✅ ZIP ready: {ZIP_PATH}")
    else:
        print("⚠️ Tidak ada file untuk di-zip.")


> ### Heaping, Per Variable for each Puskesmas

In [ ]:
import os
import re
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shutil
from matplotlib.backends.backend_pdf import PdfPages

# ===============================================================
# CONFIGURATION
# ===============================================================
COL_PKM = "puskesmas_names"
COL_PETUGAS = "perawat_bidan_nutrisionist_sanitarian"

VARS = [
    "sys", "dia", "heart_rate", "weight_kg",
    "height_cm", "resp_rate_per_minute", "body_temp_c"
]

PUSKESMAS_LIST = [
    "Puskesmas Bobotsari",
    "Puskesmas Serayu Larangan",
    "Puskesmas Karang Tengah",
    "Puskesmas Kaligondang",
    "Puskesmas Bojong",
    "Puskesmas Karangjambu",
    "Puskesmas Bojongsari",
    "Puskesmas Karanganyar",
    "Puskesmas Purbalingga",
    "Puskesmas Padamara",
    "Puskesmas Rembang",
    "Puskesmas Pengadegan"
]
OUTPUT_DIR = "Visualization of last-digit histogram"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ===============================================================
# CLEANUP FILE HEAPING LAMA (SEKALI JALAN)
# ===============================================================
for f in glob.glob(os.path.join(OUTPUT_DIR, "**/*_heaping_by_*.pdf"), recursive=True):
    os.remove(f)

# ===============================================================
# HELPERS
# ===============================================================
def normalize_worker(s: pd.Series) -> pd.Series:
    def clean(x):
        if pd.isna(x) or str(x).strip() == "":
            return "TIDAK ADA PETUGAS"
        x = str(x).upper().strip()
        x = re.sub(r"[\.,()]", "", x)
        x = re.sub(r"\s+", " ", x)
        return x
    return s.apply(clean)

def short_label(name: str) -> str:
    p = str(name).title().split()
    return f"{p[0]} {p[1][:3]}" if len(p) >= 2 else p[0]

def make_lastdigit_distribution(df, col):
    df = df.copy()
    df[col] = pd.to_numeric(df[col], errors="coerce")
    df = df.dropna(subset=[col])
    if df.empty:
        return pd.DataFrame(columns=["last_digit", "count", "pct"]), 0

    if col == "body_temp_c":
        df["last_digit"] = ((df[col] * 10).astype(int) % 10)
    else:
        df["last_digit"] = df[col].round().astype(int) % 10

    s = df["last_digit"].value_counts().sort_index().reset_index()
    s.columns = ["last_digit", "count"]
    total = s["count"].sum()
    s["pct"] = 100 * s["count"] / total
    return s, total

# ===============================================================
# BASE HISTOGRAM
# ===============================================================
def plot_digit_distribution(summary, total, title, save_path):
    summary = summary.sort_values("last_digit").reset_index(drop=True)
    plt.figure(figsize=(12, 6))
    bars = plt.bar(summary["last_digit"], summary["pct"])
    plt.title(title, fontsize=16)
    plt.xlabel("Last Digit (0–9)")
    plt.ylabel("Percentage (%)")
    plt.ylim(0, 100)
    plt.xticks(range(10))

    for bar, row in zip(bars, summary.itertuples(index=False)):
        plt.text(
            bar.get_x() + bar.get_width() / 2,
            bar.get_height() + 1,
            f"{int(row.count)} / {int(total)} = {row.pct:.1f}%",
            ha="center", va="bottom", fontsize=9, rotation=90
        )

    plt.tight_layout()
    plt.savefig(save_path, format="pdf", bbox_inches="tight")
    plt.close()

# ===============================================================
# WORKER HEAPING (SINGLE PAGE)
# ===============================================================
def plot_worker_heaping(df_src, value_col, pkm_name, pdf, mode="percentage"):
    df = df_src[df_src[COL_PKM] == pkm_name].copy()
    df[value_col] = pd.to_numeric(df[value_col], errors="coerce")
    df = df.dropna(subset=[value_col])
    if df.empty:
        return

    df["full_name"] = normalize_worker(df[COL_PETUGAS])
    df["short_name"] = df["full_name"].apply(short_label)

    if value_col == "body_temp_c":
        df["last_digit"] = ((df[value_col] * 10).astype(int) % 10)
    else:
        df["last_digit"] = df[value_col].round().astype(int) % 10

    summary_pkm, _ = make_lastdigit_distribution(df, value_col)
    heaping_digits = summary_pkm.loc[summary_pkm["pct"] > 15, "last_digit"].tolist()
    if not heaping_digits:
        return

    totals = df.groupby("full_name")[value_col].size().astype(int)

    counts = (
        df[df["last_digit"].isin(heaping_digits)]
        .groupby(["full_name", "last_digit"])[value_col]
        .size()
        .reset_index(name="count")
    )

    counts["total_all"] = counts["full_name"].map(totals)
    counts["pct"] = 100 * counts["count"] / counts["total_all"]

    name_map = df.drop_duplicates("full_name").set_index("full_name")["short_name"].to_dict()

    worker_order = totals.sort_values(ascending=False).index.tolist()
    digits = sorted(heaping_digits)
    workers = [w for w in worker_order if w in counts["full_name"].unique()]

    n_workers = len(workers)
    n_digits = len(digits)
    bar_width = 0.8 / n_digits
    x_base = np.arange(n_workers)
    fig_h = max(6, min(12, 4 + 0.25 * n_workers))
    fig, ax = plt.subplots(figsize=(max(8, n_workers), fig_h))

    for j, d in enumerate(digits):
        x = x_base + (j - (n_digits - 1) / 2) * bar_width
        vals = []
        for w in workers:
            row = counts[(counts["full_name"] == w) & (counts["last_digit"] == d)]
            if row.empty:
                vals.append(0)
            else:
                vals.append(
                    row["pct"].values[0] if mode == "percentage"
                    else row["count"].values[0]
                )
        bars = ax.bar(x, vals, width=bar_width, label=str(d))

        # ===== TEXT BAR (FIXED & BACK) =====
        for rect, w in zip(bars, workers):
            row = counts[(counts["full_name"] == w) & (counts["last_digit"] == d)]
            if row.empty:
                continue
            c = int(row["count"].values[0])
            tot = int(row["total_all"].values[0])
            pct = float(row["pct"].values[0])

            label = f"{c} / {tot} = {pct:.1f}%" if mode == "percentage" else f"{c}"

            # ===== threshold 90% logic =====
            y_max = 100 if mode == "percentage" else ax.get_ylim()[1]
            h = rect.get_height()
            ratio = h / y_max if y_max > 0 else 0

            if ratio >= 0.9:
                # text di DALAM bar
                ax.text(
                    rect.get_x() + rect.get_width() / 2,
                    h / 2,
                    label,
                    ha="center",
                    va="center",
                    fontsize=8,
                    rotation=90,
                    color="white"
                )
            else:
                # text di ATAS bar
                ax.text(
                    rect.get_x() + rect.get_width() / 2,
                    h + (0.5 if mode == "percentage" else 0.1),
                    label,
                    ha="center",
                    va="bottom",
                    fontsize=8,
                    rotation=90,
                    color="black"
                )


    ax.set_title(
        f"Heaping digits {digits} for {value_col.replace('_',' ').title()} "
        f"— {pkm_name} {'by Percentage' if mode=='percentage' else 'by Total'}"
    )
    ax.set_ylabel(
        "Percentage within Health Worker (%)" if mode == "percentage" else "Total Count"
    )
    if mode == "percentage":
        ax.set_ylim(0, 100)

    ax.set_xticks(x_base)
    ax.set_xticklabels([name_map[w] for w in workers], rotation=45, ha="right")
    ax.legend(title="Digit")
    fig.tight_layout()
    pdf.savefig(fig)
    plt.close(fig)

# ===============================================================
# MAIN PIPELINE
# ===============================================================
for var in VARS:
    print(f"\n===== Processing variable: {var} =====")

    summary_all, total_all = make_lastdigit_distribution(data, var)
    if not summary_all.empty:
        save_dir_all = os.path.join(OUTPUT_DIR, "All puskesmas")
        os.makedirs(save_dir_all, exist_ok=True)
        plot_digit_distribution(
            summary_all,
            total_all,
            f"Overall Distribution of Last Digits (0–9) for {var.replace('_',' ').title()} – All Puskesmas",
            os.path.join(save_dir_all, f"{var}_overall.pdf")
        )

    for pkm in PUSKESMAS_LIST:
        df_pkm = data[data[COL_PKM] == pkm]
        if df_pkm.empty:
            continue

        summary_pkm, total_pkm = make_lastdigit_distribution(df_pkm, var)
        if summary_pkm.empty:
            continue

        save_dir_pkm = os.path.join(OUTPUT_DIR, pkm)
        os.makedirs(save_dir_pkm, exist_ok=True)

        plot_digit_distribution(
            summary_pkm,
            total_pkm,
            f"Distribution of Last Digits (0–9) for {var.replace('_',' ').title()} – {pkm}",
            os.path.join(save_dir_pkm, f"{var}_overall.pdf")
        )

        if (summary_pkm["pct"] > 15).any():
            out_pdf = os.path.join(save_dir_pkm, f"{var}_heaping.pdf")
            with PdfPages(out_pdf) as pdf:
                plot_worker_heaping(data, var, pkm, pdf, "percentage")
                plot_worker_heaping(data, var, pkm, pdf, "total")

print("\n✅ Semua visualisasi selesai dibuat!")

# ===============================================================
# ZIP OUTPUT
# ===============================================================
shutil.make_archive(OUTPUT_DIR, "zip", OUTPUT_DIR)
print(f"📦 Output dizip ke {OUTPUT_DIR}.zip")
